In [31]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)

In [32]:
import time

driver.get('https://www.abf.gov.au/help-and-support/notices/australian-customs-notices')
time.sleep(2)

In [33]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [34]:
temp = soup.find('ul', attrs={'class' : 'pagination'}).find_all("li")
# len(temp)

In [35]:
from selenium.webdriver.common.by import By
import re

In [36]:
dates = []
titles = []
links = []
last_page = len(temp)

for i in range(2,last_page):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    date_rows = soup.select('div.col-sm-3 span')
    title_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-header')})
    link_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-content')})
    
    for row in date_rows:
        date = row.text
        dates.append(date)

    for row in title_rows:
        tds = row.find_all('td')
        title = tds[1].get_text().replace('Press Enter to show more details.', '')  
        titles.append(title)

    for row in link_rows:
        link = 'https://www.abf.gov.au' + row.a['href']
        links.append(link)
    
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(2)
    
    elem = driver.find_element(By.LINK_TEXT,str(i))
    elem.click()
    time.sleep(1)

In [37]:
soup = BeautifulSoup(driver.page_source, 'lxml')
    
date_rows = soup.select('div.col-sm-3 span')
title_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-header')})
link_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-content')})
    
for row in date_rows:
    date = row.text
    dates.append(date)

for row in title_rows:
    tds = row.find_all('td')
    title = tds[1].get_text().replace('Press Enter to show more details.', '')  
    titles.append(title)

for row in link_rows:
    link = 'https://www.abf.gov.au' + row.a['href']
    links.append(link)

In [38]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
import io
import urllib.request
import requests


def pdf_to_text(pdf_file):
    text_memory_file = io.StringIO()

    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, text_memory_file, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    # get first 3 pages of the pdf file
    for page in PDFPage.get_pages(pdf_file, pagenos=(0,0)):
        interpreter.process_page(page)
    text = text_memory_file.getvalue()
    text_memory_file.close()
    return text



In [ ]:
texts = []
start = 0
finish = len(dates)


# online pdf to text by requests
# response = requests.get("https://www.abf.gov.au/help-and-support-subsite/CustomsNotices/2022-10.pdf")
# pdf_memory_file = io.BytesIO()
# pdf_memory_file.write(response.content)
# print(pdf_to_text(pdf_memory_file))


for i in range(start, finish): # 100개씩 끊어서 돌린 뒤 texts에 추가, 총 1683개의 링크, 17번 돌리기
    url = links[i]
    response = requests.get(url)
    my_raw_data = response.content
    
    # online pdf to text by requests
    response = requests.get(url)
    pdf_memory_file = io.BytesIO()
    pdf_memory_file.write(response.content)
    texts.append(pdf_to_text(pdf_memory_file))


The PDF <_io.BytesIO object at 0x000001F5B8F5C450> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


In [ ]:
import pandas as pd

df = pd.DataFrame()
df['date'] = pd.to_datetime(dates)
df['link'] = links
df['title'] = titles
df['text'] = texts

df

In [ ]:
len(df)

In [25]:
df.to_csv('호주_세관_전체.csv', mode='w', encoding='utf-8-sig', index = False)